In [5]:
%pip install ta

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pandas_ta as ta
from ta.trend import ADXIndicator, SMAIndicator
from ta.momentum import StochasticOscillator
from ta.volatility import BollingerBands
from ta.utils import dropna

# Load and preprocess data
def load_data(file_path):
    """Load and preprocess OHLCV data."""
    data = pd.read_csv(file_path)
    data['datetime'] = pd.to_datetime(data['timestamp'], unit='s')
    data.set_index('datetime', inplace=True)
    data.drop(['timestamp'], axis=1, inplace=True)
    data.fillna(method='ffill', inplace=True)  # Handle missing values
    return dropna(data)

# Add Technical Indicators
def add_indicators(data):
    """Add ADX, Bollinger Bands, Stochastic Oscillator, and Supertrend."""
    adx = ADXIndicator(high=data['high'], low=data['low'], close=data['close'], window=14)
    data['adx'] = adx.adx()
    data['adx_pos'] = adx.adx_pos()
    data['adx_neg'] = adx.adx_neg()

    bb = BollingerBands(close=data['close'], window=20, window_dev=2)
    data['bb_upper'] = bb.bollinger_hband()
    data['bb_lower'] = bb.bollinger_lband()
    data['bb_mid'] = bb.bollinger_mavg()

    stoch = StochasticOscillator(high=data['high'], low=data['low'], close=data['close'], window=14, smooth_window=3)
    data['stoch_k'] = stoch.stoch()
    data['stoch_d'] = stoch.stoch_signal()

    atr = SMAIndicator(close=data['close'], window=10).sma_indicator()
    multiplier = 3
    data['supertrend'] = np.where((data['close'] > data['high'] - multiplier * atr), 1, -1)

    return data

# Plot Heatmap
def plot_heatmap(data):
    """Generate a correlation heatmap for OHLCV data with technical indicators."""
    plt.figure(figsize=(12, 8))
    sns.heatmap(data.corr(), annot=True, cmap="coolwarm", fmt=".2f")
    plt.title("Correlation Heatmap")
    plt.show()

# Preprocess for Random Forest
def preprocess_data(data, target='close', sequence_length=60):
    """Preprocess data for Random Forest model."""
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data[[target]].values)

    X, y = [], []
    for i in range(sequence_length, len(scaled_data)):
        X.append(scaled_data[i-sequence_length:i, 0])
        y.append(scaled_data[i, 0])
    return np.array(X), np.array(y), scaler

# Random Forest Model
def create_random_forest():
    """Create a Random Forest model."""
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    return model

# Main Execution
if __name__ == "__main__":
    # Load and preprocess data
    file_path = "BTC_2019_2023_1d.csv"  # Replace with your file path
    data = load_data(file_path)
    data = add_indicators(data)

    # Plot Heatmap
    plot_heatmap(data)

    # Preprocess for Random Forest
    feature = 'close'
    sequence_length = 60
    X, y, scaler = preprocess_data(data, feature, sequence_length)

    # Train-Test Split
    split = int(len(X) * 0.8)
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    # Train Random Forest
    model = create_random_forest()
    model.fit(X_train, y_train)

    # Predictions
    predictions = model.predict(X_test)
    predictions_inverse = scaler.inverse_transform(predictions.reshape(-1, 1))
    y_test_inverse = scaler.inverse_transform(y_test.reshape(-1, 1))

    # Evaluate Model
    mse = mean_squared_error(y_test_inverse, predictions_inverse)
    mae = mean_absolute_error(y_test_inverse, predictions_inverse)
    print(f"Model MSE: {mse}")
    print(f"Model MAE: {mae}")

    # Plot Predictions
    plt.figure(figsize=(12, 6))
    plt.plot(y_test_inverse, label="Actual Prices")
    plt.plot(predictions_inverse, label="Predicted Prices", alpha=0.7)
    plt.title("Random Forest Price Prediction vs Actual")
    plt.legend()
    plt.show()


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


KeyError: 'timestamp'